<div >
<img src = "../banner.jpg" />
</div>

# Desbalance de Classes

In [1]:
#Cargar librerías 
require("pacman")
p_load("tidyverse")


Loading required package: pacman



In [2]:
#Leer los datos 
credit <- readRDS(url("https://github.com/ignaciomsarmiento/datasets/blob/main/credit_class.rds?raw=true"))
#mutacion de factores
credit<-credit %>% mutate(Default=factor(Default,levels=c(0,1),labels=c("No","Si")),
                          history=factor(history,levels=c("good","poor","terrible"),labels=c("buena","mala","terrible")),
                          foreign=factor(foreign,levels=c("foreign","german"),labels=c("extranjero","aleman")),
                          purpose=factor(purpose,levels=c("newcar","usedcar","goods/repair","edu", "biz" ),labels=c("auto_nuevo","auto_usado","bienes","educacion","negocios")))         

head(credit)

,Default,duration,amount,installment,age,history,purpose,foreign,rent
,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,No,6,1169,4,67,terrible,bienes,extranjero,FALSE
2,Si,48,5951,2,22,mala,bienes,extranjero,FALSE
3,No,12,2096,2,49,terrible,educacion,extranjero,FALSE
4,No,42,7882,2,45,mala,bienes,extranjero,FALSE
5,Si,24,4870,3,53,mala,auto_nuevo,extranjero,FALSE
6,No,36,9055,2,35,mala,educacion,extranjero,FALSE


In [3]:
p_load("caret")
dmy <- dummyVars(" ~ .", data = credit)
head(dmy)

$call
dummyVars.default(formula = " ~ .", data = credit)

$form
~.
<environment: 0x7f8b43835610>

$vars
[1] "Default"     "duration"    "amount"      "installment" "age"        
[6] "history"     "purpose"     "foreign"     "rent"       

$facVars
[1] "Default" "history" "purpose" "foreign" "rent"   

$lvls
$lvls$Default
[1] "No" "Si"

$lvls$history
[1] "buena"    "mala"     "terrible"

$lvls$purpose
[1] "auto_nuevo" "auto_usado" "bienes"     "educacion"  "negocios"  

$lvls$foreign
[1] "extranjero" "aleman"    

$lvls$rent
[1] NA      "FALSE" "TRUE" 


$sep
[1] "."


In [4]:
credit <- data.frame(predict(dmy, newdata = credit))
credit<- credit  %>% mutate(Default=factor(Default.Si,levels=c(0,1),labels=c("No","Si")))
head(credit)

,Default.No,Default.Si,duration,amount,installment,age,history.buena,history.mala,history.terrible,purpose.auto_nuevo,purpose.auto_usado,purpose.bienes,purpose.educacion,purpose.negocios,foreign.extranjero,foreign.aleman,rent.NA,rent.FALSE,rent.TRUE,Default
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,0,6,1169,4,67,0,0,1,0,0,1,0,0,1,0,0,1,0,No
2,0,1,48,5951,2,22,0,1,0,0,0,1,0,0,1,0,0,1,0,Si
3,1,0,12,2096,2,49,0,0,1,0,0,0,1,0,1,0,0,1,0,No
4,1,0,42,7882,2,45,0,1,0,0,0,1,0,0,1,0,0,1,0,No
5,0,1,24,4870,3,53,0,1,0,1,0,0,0,0,1,0,0,1,0,Si
6,1,0,36,9055,2,35,0,1,0,0,0,0,1,0,1,0,0,1,0,No


In [5]:
## First, split the training set 
set.seed(156)
split1 <- createDataPartition(credit$Default, p = .7)[[1]]
length(split1)
head(split1, n=20)


[1] 700

[1]  1  2  3  4  5  8 11 12 13 14 15 17 18 19 20 21 22 23 25 26

In [6]:
other     <- credit[-split1,]
training  <- credit[ split1,]

In [7]:
set.seed(934)
split2 <- createDataPartition(other$Default, p = 1/3)[[1]]
evaluation  <- other[ split2,]
testing     <- other[-split2,]

In [8]:
dim(training)
dim(testing)
dim(evaluation)

[1] 700  20

[1] 200  20

[1] 100  20

In [9]:
#Logit

In [10]:
fiveStats <- function(...) c(twoClassSummary(...), defaultSummary(...))
ctrl<- trainControl(method = "cv",
                     number = 5,
                     summaryFunction = fiveStats,
                     classProbs = TRUE,
                     verbose=FALSE,
                     savePredictions = T)
#logit
set.seed(1410)
mylogit_caret <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = training, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial", 
                       metric = 'ROC')




mylogit_caret

Generalized Linear Model 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results:

  ROC        Sens       Spec       Accuracy   Kappa    
  0.7308066  0.9102041  0.3047619  0.7285714  0.2456281


## Model Tunning: Lasso

### Accuracy

In [11]:
#Lasso
lambda_grid <- 10^seq(-4, 0.01, length = 10) #en la practica se suele usar una grilla de 200 o 300


set.seed(1410)
mylogit_lasso_acc <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE,
  data = training, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "Accuracy",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_acc

glmnet 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec        Accuracy   Kappa     
  0.0001000000  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0002789687  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0007782356  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0021710342  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0060565070  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0168957618  0.7327017  0.9142857  0.27619048  0.7228571  0.22125732
  0.0471338954  0.7335763  0.9326531  0.26190476  0.7314286  0.23129637
  0.1314888384  0.7347425  0.9571429  0.14285714  0.7128571  0.12505390
  0.3668127682  0.7336249  0.9918367  0.02380952  0.7014286  0.02124838
  1.0232929923  0.7298834  1.0000000  0.0

### ROC

In [12]:
set.seed(1410)
mylogit_lasso_roc <- train(
 Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
  data = training, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_roc

glmnet 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 560, 560 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec        Accuracy   Kappa     
  0.0001000000  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0002789687  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0007782356  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0021710342  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0060565070  0.7318756  0.9142857  0.28571429  0.7257143  0.23176719
  0.0168957618  0.7327017  0.9142857  0.27619048  0.7228571  0.22125732
  0.0471338954  0.7335763  0.9326531  0.26190476  0.7314286  0.23129637
  0.1314888384  0.7347425  0.9571429  0.14285714  0.7128571  0.12505390
  0.3668127682  0.7336249  0.9918367  0.02380952  0.7014286  0.02124838
  1.0232929923  0.7298834  1.0000000  0.0

## Alternative Cutoffs

In [13]:

evalResults <- data.frame(Default = evaluation$Default)

evalResults$Roc <- predict(mylogit_lasso_roc,
                              newdata = evaluation,
                              type = "prob")[,1]


In [14]:
p_load("pROC")
rfROC <- roc(evalResults$Default, evalResults$Roc, levels = rev(levels(evalResults$Default)))
rfROC

Setting direction: controls < cases




Call:
roc.default(response = evalResults$Default, predictor = evalResults$Roc,     levels = rev(levels(evalResults$Default)))

Data: evalResults$Roc in 30 controls (evalResults$Default Si) < 70 cases (evalResults$Default No).
Area under the curve: 0.7871

In [15]:
rfThresh <- coords(rfROC, x = "best", best.method = "closest.topleft")
rfThresh

threshold,specificity,sensitivity
<dbl>,<dbl>,<dbl>
0.662454,0.7,0.8142857


In [16]:
evalResults<-evalResults %>% mutate(hat_def_05=ifelse(evalResults$Roc>0.5,"Si","No"),
                                    hat_def_rfThresh=ifelse(evalResults$Roc>rfThresh$threshold,"Si","No"))

In [17]:
with(evalResults,table(Default,hat_def_05))


       hat_def_05
Default No Si
     No  1 69
     Si  5 25

In [18]:
with(evalResults,table(Default,hat_def_rfThresh))

       hat_def_rfThresh
Default No Si
     No 13 57
     Si 21  9

## Remuestreo

<div >
<img src = "sampling_methods.png" />
</div>

### Up Sampling

In [19]:
set.seed(1103)
upSampledTrain <- upSample(x = training,
                           y = training$Default,
                           ## keep the class variable name the same:
                           yname = "Default")
dim(training)

dim(upSampledTrain)

table(upSampledTrain$Default)


[1] 700  20

[1] 980  21


 No  Si 
490 490 

In [20]:

set.seed(1410)
mylogit_lasso_upsample <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
  data = upSampledTrain, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_upsample


glmnet 

980 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 784, 784, 784, 784, 784 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec       Accuracy   Kappa    
  0.0001000000  0.7475427  0.7122449  0.6530612  0.6826531  0.3653061
  0.0002789687  0.7475427  0.7122449  0.6530612  0.6826531  0.3653061
  0.0007782356  0.7475427  0.7122449  0.6530612  0.6826531  0.3653061
  0.0021710342  0.7475427  0.7122449  0.6530612  0.6826531  0.3653061
  0.0060565070  0.7475427  0.7122449  0.6530612  0.6826531  0.3653061
  0.0168957618  0.7478759  0.7102041  0.6551020  0.6826531  0.3653061
  0.0471338954  0.7480633  0.7081633  0.6489796  0.6785714  0.3571429
  0.1314888384  0.7482716  0.7142857  0.6469388  0.6806122  0.3612245
  0.3668127682  0.7454810  0.7061224  0.6367347  0.6714286  0.3428571
  1.0232929923  0.7438984  0.7163265  0.6387755  0.6775510  0

### Down Sampling

In [21]:

set.seed(1103)
downSampledTrain <- downSample(x = training,
                           y = training$Default,
                           ## keep the class variable name the same:
                           yname = "Default")
dim(training)

dim(downSampledTrain)

table(downSampledTrain$Default)





[1] 700  20

[1] 420  21


 No  Si 
210 210 

In [22]:
set.seed(1410)
mylogit_lasso_downsample <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       rent.TRUE, 
  data = downSampledTrain, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_downsample

glmnet 

420 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 336, 336, 336, 336, 336 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec       Accuracy   Kappa    
  0.0001000000  0.7380952  0.6952381  0.6476190  0.6714286  0.3428571
  0.0002789687  0.7380952  0.6952381  0.6476190  0.6714286  0.3428571
  0.0007782356  0.7380952  0.6952381  0.6476190  0.6714286  0.3428571
  0.0021710342  0.7380952  0.6952381  0.6476190  0.6714286  0.3428571
  0.0060565070  0.7380952  0.6952381  0.6476190  0.6714286  0.3428571
  0.0168957618  0.7390023  0.7000000  0.6476190  0.6738095  0.3476190
  0.0471338954  0.7400227  0.7000000  0.6476190  0.6738095  0.3476190
  0.1314888384  0.7394558  0.7000000  0.6476190  0.6738095  0.3476190
  0.3668127682  0.7404762  0.7095238  0.6428571  0.6761905  0.3523810
  1.0232929923  0.7366213  0.7142857  0.6428571  0.6785714  0

### SMOTE

<div >
<img src = "smote.png" />
</div>

In [23]:
p_load("smotefamily")
predictors<-c("duration","amount","installment","age",
                       "history.buena","history.mala",
                       "purpose.auto_nuevo","purpose.auto_usado","purpose.bienes","purpose.educacion",
                       "foreign.extranjero",
                       "rent.TRUE")
head( training[predictors])
smote_output = SMOTE(X = training[predictors],
                     target = training$Default)
oversampled_data = smote_output$data
table(training$Default)
table(oversampled_data$class)

,duration,amount,installment,age,history.buena,history.mala,purpose.auto_nuevo,purpose.auto_usado,purpose.bienes,purpose.educacion,foreign.extranjero,rent.TRUE
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6,1169,4,67,0,0,0,0,1,0,1,0
2,48,5951,2,22,0,1,0,0,1,0,1,0
3,12,2096,2,49,0,0,0,0,0,1,1,0
4,42,7882,2,45,0,1,0,0,1,0,1,0
5,24,4870,3,53,0,1,1,0,0,0,1,0
8,36,6948,2,35,0,1,0,1,0,0,1,1



 No  Si 
490 210 


 No  Si 
490 420 

In [24]:
set.seed(1410)
mylogit_lasso_smote<- train(class~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       rent.TRUE,
  data = oversampled_data, 
  method = "glmnet",
  trControl = ctrl,
  family = "binomial", 
  metric = "ROC",
  tuneGrid = expand.grid(alpha = 0,lambda=lambda_grid), 
  preProcess = c("center", "scale")
)
mylogit_lasso_smote

glmnet 

910 samples
 12 predictor
  2 classes: 'No', 'Si' 

Pre-processing: centered (12), scaled (12) 
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 728, 728, 728, 728, 728 
Resampling results across tuning parameters:

  lambda        ROC        Sens       Spec       Accuracy   Kappa    
  0.0001000000  0.7575316  0.7591837  0.5857143  0.6791209  0.3481159
  0.0002789687  0.7575316  0.7591837  0.5857143  0.6791209  0.3481159
  0.0007782356  0.7575316  0.7591837  0.5857143  0.6791209  0.3481159
  0.0021710342  0.7575316  0.7591837  0.5857143  0.6791209  0.3481159
  0.0060565070  0.7575316  0.7591837  0.5857143  0.6791209  0.3481159
  0.0168957618  0.7579932  0.7612245  0.5785714  0.6769231  0.3432096
  0.0471338954  0.7580175  0.7632653  0.5666667  0.6725275  0.3335049
  0.1314888384  0.7580175  0.7897959  0.5571429  0.6824176  0.3517090
  0.3668127682  0.7569242  0.8408163  0.4976190  0.6824176  0.3460137
  1.0232929923  0.7534742  0.9040816  0.3309524  0.6395604  0

## Compación

In [25]:
testResults <- data.frame(Default = testing$Default)

testResults$logit<- predict(mylogit_caret,
                           newdata = testing,
                           type = "prob")[,1]
testResults$lasso<- predict(mylogit_lasso_roc,
                           newdata = testing,
                           type = "prob")[,1]

testResults$lasso_thresh<- predict(mylogit_lasso_roc,
                           newdata = testing,
                           type = "prob")[,1]

testResults$lasso_upsample<- predict(mylogit_lasso_upsample,
                           newdata = testing,
                           type = "prob")[,1]

testResults$mylogit_lasso_downsample<- predict(mylogit_lasso_downsample,
                           newdata = testing,
                           type = "prob")[,1]

testResults$mylogit_lasso_smote<- predict(mylogit_lasso_smote,
                           newdata = testing,
                           type = "prob")[,1]

testResults<-testResults %>% 
              mutate(logit=ifelse(logit>0.5,"Si","No"),
                     lasso=ifelse(lasso>0.5,"Si","No"),
                     lasso_thresh=ifelse(lasso_thresh>rfThresh$threshold,"Si","No"),
                     lasso_upsample=ifelse(lasso_upsample>0.5,"Si","No"),
                     mylogit_lasso_downsample=ifelse(mylogit_lasso_downsample>0.5,"Si","No"),
                     mylogit_lasso_smote=ifelse(mylogit_lasso_smote>0.5,"Si","No"),
                     )

In [26]:
with(testResults,table(Default,logit))

       logit
Default  No  Si
     No  18 122
     Si  17  43

In [27]:
with(testResults,table(Default,lasso))

       lasso
Default  No  Si
     No   8 132
     Si   9  51

In [28]:
with(testResults,table(Default,lasso_thresh))

       lasso_thresh
Default  No  Si
     No  36 104
     Si  29  31

In [29]:
with(testResults,table(Default,lasso_upsample))

       lasso_upsample
Default No Si
     No 46 94
     Si 38 22

In [30]:
with(testResults,table(Default,mylogit_lasso_downsample))

       mylogit_lasso_downsample
Default No Si
     No 53 87
     Si 36 24

In [31]:
with(testResults,table(Default,mylogit_lasso_smote))

       mylogit_lasso_smote
Default  No  Si
     No  34 106
     Si  31  29